In [1]:
from langchain_community.retrievers import BM25Retriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from dotenv import load_dotenv

load_dotenv()


c:\RAG\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

In [2]:
%pip install rank_bm25

In [3]:
# ──────────────────────────────────────────────────────────────────
# SETUP: Create our sample company data
# ──────────────────────────────────────────────────────────────────

chunks = [
    "Microsoft acquired GitHub for 7.5 billion dollars in 2018.",
    "Tesla Cybertruck production ramp begins in 2024.",
    "Google is a large technology company with global operations.",
    "Tesla reported strong quarterly results. Tesla continues to lead in electric vehicles. Tesla announced new manufacturing facilities.",
    "SpaceX develops Starship rockets for Mars missions.",
    "The tech giant acquired the code repository platform for software development.",
    "NVIDIA designs Starship architecture for their new GPUs.",
    "Tesla Tesla Tesla financial quarterly results improved significantly.",
    "Cybertruck reservations exceeded company expectations.",
    "Microsoft is a large technology company with global operations.", 
    "Apple announced new iPhone features for developers.",
    "The apple orchard harvest was excellent this year.",
    "Python programming language is widely used in AI.",
    "The python snake can grow up to 20 feet long.",
    "Java coffee beans are imported from Indonesia.", 
    "Java programming requires understanding of object-oriented concepts.",
    "Orange juice sales increased during winter months.",
    "Orange County reported new housing developments."
]

In [4]:
# Convert to Document objects for LangChain
documents = [Document(page_content=chunk, metadata={"source": f"chunk_{i}"}) for i, chunk in enumerate(chunks)]

print("Sample Data:")
for i, chunk in enumerate(chunks, 1):
    print(f"{i}. {chunk}")

print("\n" + "="*80)

Sample Data:
1. Microsoft acquired GitHub for 7.5 billion dollars in 2018.
2. Tesla Cybertruck production ramp begins in 2024.
3. Google is a large technology company with global operations.
4. Tesla reported strong quarterly results. Tesla continues to lead in electric vehicles. Tesla announced new manufacturing facilities.
5. SpaceX develops Starship rockets for Mars missions.
6. The tech giant acquired the code repository platform for software development.
7. NVIDIA designs Starship architecture for their new GPUs.
8. Tesla Tesla Tesla financial quarterly results improved significantly.
9. Cybertruck reservations exceeded company expectations.
10. Microsoft is a large technology company with global operations.
11. Apple announced new iPhone features for developers.
12. The apple orchard harvest was excellent this year.
13. Python programming language is widely used in AI.
14. The python snake can grow up to 20 feet long.
15. Java coffee beans are imported from Indonesia.
16. Java pr

1. Vector Retriever (Semantic Search/Dense Retrieval)

In [5]:
print("Setting up Vector Retriever...")
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_metadata={"hnsw:space": "cosine"}
)

Setting up Vector Retriever...


In [6]:
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# Test semantic search
test_query = "space exploration company" #works in vector search but wouldn't work with keyword search

print(f"Testing: '{test_query}'")
test_docs = vector_retriever.invoke(test_query)
for doc in test_docs:
    print(f"Found: {doc.page_content}")

Testing: 'space exploration company'
Found: SpaceX develops Starship rockets for Mars missions.
Found: Google is a large technology company with global operations.


2. BM25 Retriever (Keyword Search/Sparse Retrieval)

In [7]:
# 2. BM25 Retriever (Keyword Search)
print("Setting up BM25 Retriever...")
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 3

Setting up BM25 Retriever...


In [8]:
# Test exact keyword matching
# test_query = "space exploration company"
# test_query = "Cybertruck"
test_query = "Tesla"

print(f"Testing: '{test_query}'")
test_docs = bm25_retriever.invoke(test_query)
for doc in test_docs:
    print(f"Found: {doc.page_content}")

Testing: 'Tesla'
Found: Tesla Tesla Tesla financial quarterly results improved significantly.
Found: Tesla reported strong quarterly results. Tesla continues to lead in electric vehicles. Tesla announced new manufacturing facilities.
Found: Tesla Cybertruck production ramp begins in 2024.


3. Hybrid Retriever (Combination)

In [9]:
#  3. Hybrid Retriever (Combination)
# 3. Hybrid Retriever (Combination) – LangChain 1.x compatible

print("Setting up Hybrid Retriever...")

from langchain_core.runnables import RunnableLambda

def hybrid_search(query: str, k: int = 4):
    vector_docs = vector_retriever.invoke(query)
    bm25_docs = bm25_retriever.invoke(query)

    scores = {}

    # Vector retriever weight = 0.7
    for rank, doc in enumerate(vector_docs):
        scores[doc.page_content] = scores.get(doc.page_content, 0) + 0.7 * (1 / (rank + 1))

    # BM25 retriever weight = 0.3
    for rank, doc in enumerate(bm25_docs):
        scores[doc.page_content] = scores.get(doc.page_content, 0) + 0.3 * (1 / (rank + 1))

    ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in ranked_docs[:k]]

hybrid_retriever = RunnableLambda(hybrid_search)

print("Setup complete!\n")


Setting up Hybrid Retriever...
Setup complete!



In [10]:
# Query 1: Mixed semantic and exact terms

# Vector search understands "purchase cost" semantically
# BM25 search finds exact "7.5 billion" 
# Hybrid should combine both strengths for best result
test_query = "purchase cost 7.5 billion"

retrieved_chunks = hybrid_retriever.invoke(test_query)
for i, doc in enumerate(retrieved_chunks, 1):
    print(f"{i}. {doc.page_content}")
print()

print("Query 1 shows how hybrid finds exact financial info using both semantic understanding and keyword matching")

AttributeError: 'str' object has no attribute 'page_content'

In [ ]:
# Query 2: Semantic concept + specific product name  

# Vector search understands "electric vehicle manufacturing"
# BM25 search finds exact "Cybertruck"
# Hybrid gets the best of both worlds

test_query = "electric vehicle manufacturing Cybertruck"

retrieved_chunks = hybrid_retriever.invoke(test_query)

for i, doc in enumerate(retrieved_chunks, 1):
    print(f"{i}. {doc.page_content}")
print()

print("Query 2 demonstrates combining product-specific terms with broader concepts")

In [ ]:
# Query 3: Where neither alone would be perfect

# "Company performance" is semantic, "Tesla" is exact keyword
# Hybrid should find the most relevant Tesla performance info

test_query = "company performance Tesla"

retrieved_chunks = hybrid_retriever.invoke(test_query)
for i, doc in enumerate(retrieved_chunks, 1):
    print(f"{i}. {doc.page_content}")
print()

print("Query 3 shows how hybrid handles mixed semantic/keyword queries better than either approach alone")

In [ ]:
# Combine the query and the relevant document contents
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama


combined_input = f"""Based on the following documents, please answer this question: {test_query}

Documents:
{chr(10).join([f"- {doc.page_content}" for doc in retrieved_chunks])}

Please provide a clear, helpful answer using only the information from these documents. If you can't find the answer in the documents, say "I don't have enough information to answer that question based on the provided documents."
"""

# Create a ChatOllama model
model = ChatOllama(model="llama3.2")

# Define the messages for the model
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content=combined_input),
]

# Invoke the model with the combined input
result = model.invoke(messages)

# Display the full result and content only
print("\n--- Generated Response ---")
# print("Full result:")
# print(result)
print("Content only:")
print(result.content)